In [2]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request

# MBTI 별 posting 모으기

In [3]:
mbti=pd.read_csv('./data/mbti_1.csv')

types = mbti.groupby('type').count()

output_file = '.\data\\training\\mbti.csv' # raw string이 아니라 '\'를 쓰려면 \\라고 해야 함
allData = []

for type in types.index:
    condition = mbti['type'] == type # condition: mbti['type']가 topfive의 원소인 type 같을 bool 조건
    ownsentence = mbti[condition]  # ownsentence: condition에 맞는 row만 filtering한 dataframe
    allData.append(ownsentence)
dataCombine = pd.concat(allData, axis=0, ignore_index=True)
dataCombine.to_csv(output_file, index=False)

types

    #input_file = r'.\data\training'
    #allFile_list = glob.glob(os.path.join(input_file, 'mbti_*'))
    #for file in allFile_list:
    #    csv = pd.read_csv(file,sep=';', encoding='iso-8859-1') # for구문으로 csv파일들을 읽어 들인다
    #    cleanMbti = csv['posts'].str.replace('[^A-Za-z\s]+', '')

,posts
type,
ENFJ,190
ENFP,675
ENTJ,231
ENTP,685
ESFJ,42
ESFP,48
ESTJ,39
ESTP,89
INFJ,1470


# Resampling (Undersampling and Oversampling)

# Tokenize and Stemming
morph analysis 기법.

In [4]:
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [5]:
mbti= dataCombine
X_data = mbti['posts']
y_data = mbti['type']

print("normalize 시작")
normalized_text = []
for sentence in X_data.tolist():
    clean_sentence = re.sub('[^A-Za-z\s]+', '',sentence.lower())
    normalized_text.append(clean_sentence)
mbti.posts = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [16]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stopwords = ['http*']
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in mbti['posts']:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        tempData.append(ps.stem(word))
        #tempData = [word for word in tempData if not word in stopwords] # 불용어 제거
    stemData.append(tempData)
print("stemming 완료")


stemming 시작
stemming 완료
['httpswwwyoutubecomwatchvplaaikvhvz', 'oi', 'went', 'through', 'a', 'break', 'up', 'some', 'month', 'ago', 'we', 'were', 'togeth', 'for', 'year', 'and', 'i', 'had', 'plan', 'my', 'life', 'around', 'that', 'relationship', 'i', 'wasnt', 'the', 'one', 'break', 'the', 'relationship', 'as', 'you', 'might', 'imagin', 'and', 'all', 'ourenfj', 'pun', 'so', 'mani', 'punswel', 'i', 'person', 'dont', 'go', 'that', 'much', 'for', 'attract', 'in', 'gener', 'but', 'i', 'can', 'see', 'you', 'have', 'the', 'will', 'to', 'chang', 'that', 'and', 'that', 'good', 'alreadi', 'may', 'i', 'ask', 'if', 'you', 'want', 'to', 'be', 'with', 'them', 'in', 'a', 'mere', 'sexualsorri', 'not', 'an', 'infp', 'but', 'im', 'realli', 'into', 'postrock', 'so', 'i', 'had', 'to', 'post', 'go', 'to', 'leav', 'thi', 'here', 'and', 'sneak', 'out', 'd', 'httpswwwyoutubecomwatchvushcobpcmwelcomewrong', 'thread', 'dthat', 'doesnt', 'sound', 'veri', 'enfj', 'i', 'think', 'x', 'id', 'never', 'act', 'cold', '

# Padding

In [37]:
import keras
from keras.preprocessing.sequence import pad_sequences

lenth=[]
for wordlist in stemData:
    lenth.append(len(wordlist))
max_len = max(lenth)
print(max_len)
X_token = pad_sequences(stemData,maxlen=max_len, value=" ")


ModuleNotFoundError: No module named 'keras'

# Model building

In [17]:
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [26]:
X_data = np.array(X_token).reshape(-1,1)
y_data = np.array(mbti['type']).reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size= 0.3, random_state=1234)
model = XGBClassifier()
model.fit(X_train, y_train)
preds = classifier.predict(X_test)

plot_importance(model)
pyplot.show()

ValueError: setting an array element with a sequence.